In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle as pkl

In [5]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [7]:
## Encoding for categorical variables
label_gender_encoder=LabelEncoder()
data['Gender']=label_gender_encoder.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [8]:
data['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [9]:
## The encoding is fine for binary but for 3+ categories it
## consideres them as values that is 2>1>0 and this is not good
## One Hot Encoding
from sklearn.preprocessing import OneHotEncoder
ohe_geography=OneHotEncoder()
geo_encoder=ohe_geography.fit_transform(data[['Geography']])

In [10]:
ohe_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded=pd.DataFrame(geo_encoder.toarray(),columns=ohe_geography.get_feature_names_out(['Geography']))

In [12]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded],axis=1)

In [13]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [14]:
with open('label_gender_encoder.pkl','wb') as file:
    pkl.dump(label_gender_encoder,file)
with open('ohe_geography.pkl','wb') as file:
    pkl.dump(ohe_geography,file)

In [ ]:
## Dividing the data into train_test_split
X=data.drop('Exited',axis=1)
y=data['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [16]:
with open('scaler.pkl','wb') as file:
    pkl.dump(scaler,file)

<h1 style="color:white">ANN Implementation</h1>

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [19]:
X_train.shape[1]

12

In [20]:
## Build ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid')  ## Output layer
])

c:\Users\Siddhant Singh\Documents\Machine_Learning\Udemy\Projects\ANN_Churn\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
import tensorflow
optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
tensorflow.losses.BinaryCrossentropy()

<LossFunctionWrapper(<function binary_crossentropy at 0x000001FB676DDE40>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [24]:
## compile the model(used for forward and backward propogation)
model.compile(optimizer=optimizer,loss="binary_crossentropy",metrics=['accuracy'])
## Alternate method for this is
## model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"]) but in this the
## bad thing is that its learning rate etc is fixed so to give it dynamic, we can use it separately.

In [26]:
## Set up the tensorboard
log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [29]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_score',mode='min',patience=5,restore_best_weights=True)

In [30]:
## Model Training
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8528 - loss: 0.3576 - val_accuracy: 0.8590 - val_loss: 0.3659
Epoch 2/100
 52/250 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8574 - loss: 0.3554

c:\Users\Siddhant Singh\Documents\Machine_Learning\Udemy\Projects\ANN_Churn\venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_score` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8579 - loss: 0.3481 - val_accuracy: 0.8610 - val_loss: 0.3422
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8619 - loss: 0.3431 - val_accuracy: 0.8575 - val_loss: 0.3481
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8620 - loss: 0.3391 - val_accuracy: 0.8610 - val_loss: 0.3545
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8621 - loss: 0.3387 - val_accuracy: 0.8575 - val_loss: 0.3375
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8629 - loss: 0.3354 - val_accuracy: 0.8605 - val_loss: 0.3371
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8636 - loss: 0.3321 - val_accuracy: 0.8590 - val_loss: 0.3369
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8630 - loss: 0.3309 - val_accuracy: 0.8620 - val_loss: 0.3395
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8633 - loss: 0.3314 - val_accuracy: 0.8630

In [31]:
model.save('model.h5') ## As h5 is compatible

In [32]:
## Load tensorboard extension
%load_ext tensorboard

In [35]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 3408), started 0:04:09 ago. (Use '!kill 3408' to kill it.)